In [2]:
import os
import pandas as pd
import tarfile
import shutil

# Column names:

txn_name, cc, ab, benchmark, duration, clients, tps, attempts, commits, start_cnt, total_cnt, 50.0, 90.0, 99.0, 99.9, max, min, zipf, experiment_id

# Remember to pause Dropbox Sync!

In [1]:
def parse_round(result_dir):
    cur_dir = os.getcwd()
    base_dir = os.path.join(cur_dir,result_dir)
    tar_lists = [f for f in os.listdir(base_dir) if f.endswith(".tgz")]
    exp_dfs = []
    for ft in tar_lists:
    # ft = tar_lists[0]
        config = ft[:-4].split('_')
        shards=int(config[3][:-6])
        cpus=int(config[4][:-4])
        reps=int(config[6][:-3])
        ft_path = os.path.join(base_dir,ft)
        # unzip tgz files first
        tar =  tarfile.open(ft_path,'r:gz')
        tar.extractall(base_dir)
        tar.close()
        arc_dir = os.path.join(base_dir,"archive")
        # parse csv files
        csvs = [os.path.join(arc_dir,f) for f in os.listdir(arc_dir) if f.endswith(".csv")]
        if not csvs:
            continue
        all_df = (pd.read_csv(f,sep=', ',skiprows=3,header=None,usecols=[0,1,2,5,6]) for f in csvs)
        df = pd.concat(all_df, ignore_index=True)
        df.drop_duplicates(inplace=True)
        # columns interested
        df.columns = ['txn_name', 'cc', 'ab', 'clients', 'tps']#, 'attempts', 'commits', 'start_cnt', 'total_cnt', '50.0', '90.0', '99.0', '99.9', 'max', 'min', 'zipf', 'experiment_id']
        df = df.groupby(['cc', 'ab','clients']).sum().reset_index()
        df['shards'] = shards
        df['cpus'] = cpus
        df['replica'] = reps
        shutil.rmtree(arc_dir)
        exp_dfs.append(df)

    df = pd.concat(exp_dfs,ignore_index=True)
    df.drop(columns=["ab"],inplace=True)
    df.drop_duplicates(['cc','clients','shards','cpus','replica'],inplace=True,ignore_index=True)
    return df

In [3]:
df1 = parse_round("round1\\janus")
df2 = parse_round("round2\\janus")
df3 = parse_round("round3\\janus")


C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [18]:
def mean_count_miss(df):
    runs = df.shape[0]
    if runs == 1:
        return -1
    else:
        return df.mean()

In [40]:
janus_index = ['cc','clients','shards','cpus','replica']
agg_df_raw = pd.concat([df1,df2,df3],axis=0).groupby(janus_index).mean().reset_index()
agg_df_indexed = agg_df_raw.set_index(janus_index)

In [44]:
agg_df_raw.shape

(3197, 6)

In [58]:
# this cell is used to find out which settings are missing
# if you are parsing partial result, you can ignore this cell
missing = []
for rep in [1,3,5,7]:
    for cpu in [1,2,3,4]:
        WH = 1
        while rep*WH/cpu<=27:
            for cli in [1,10,20,40,80,100,200,500,1000,2000,4000]:
                for cc in ['Janus','Tapir','2PL','OCC']:
                    try:
                        agg_df_indexed.loc[(cc,cli,WH,cpu,rep)]
                    except KeyError:
                        missing.append(pd.DataFrame.from_records([{'cc':cc,'clients':cli,'shards':WH,'cpus':cpu,'replica':rep,'tps':-1}]))
            WH = WH * 2

In [59]:
miss_df = pd.concat(missing,axis=0).reset_index(drop=True)
agg_df = pd.concat([miss_df,agg_df_raw],axis=0).reset_index(drop=True)


In [61]:
agg_df.to_csv('result1_avg/janus.csv',index=False)